In [2]:
import pandas as pd
import datetime
import numpy as np
import datetime
from sklearn.linear_model import LogisticRegression
epoch = datetime.datetime.utcfromtimestamp(0)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [3]:
df = pd.read_csv('data_train.csv', parse_dates=['time_exit', 'time_entry'])

In [4]:
def unix_time_millis(dt):
    return (dt - epoch).total_seconds() * 1000.0

In [5]:
#encoding time
df_new_entry = df['time_entry'].apply(unix_time_millis)
df_new_exit = df['time_exit'].apply(unix_time_millis)

In [6]:
df['time_entry'] = df_new_entry
df['time_exit'] = df_new_exit

In [7]:
def classify_city_center(row):
    if (row['x_exit'] > 3750901.5068) and  (row['x_exit'] < 3770901.5068) and (row['y_exit'] > -19268905.6133) and (row['y_exit'] < -19208905.6133):
        return 1
    else:
        return 0

In [8]:
y = np.zeros(814262)
df['y'] = y 

In [9]:
df['y'] = df.apply(classify_city_center, axis=1)
df_clean = df.dropna(subset=['time_entry','time_exit','x_entry','y_entry'])

In [21]:
y = np.array(df['y'])

In [10]:
X = df_clean.iloc[:, [3,4,8,9]].values

In [11]:
df_0 = df_clean[df_clean['y']==0]
df_1 = df_clean[df_clean['y']==1]
X_0 = df_0.iloc[:, [3,4,8,9]]
X_1 = df_1.iloc[:, [3,4,8,9]]
y_0 = df_0['y']
y_1 = df_1['y']

In [12]:
from sklearn.model_selection import train_test_split
X_0_keep, X_0_dispose, y_0_keep, y_0_dispose = train_test_split(X_0, y_0, test_size = 0.5, random_state = 0)
X = pd.concat([X_0_keep, X_1])
y = pd.concat([y_0_keep, y_1])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [41]:
len(X)


528737

In [19]:
import xgboost as xgb
# specify parameters via map
classifier = xgb.XGBClassifier(max_depth=10)
classifier.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=100, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [20]:
y_predicted= classifier.predict(X_test)
accuracy_score(y_test,y_predicted)

0.923478458221432

# PreProcessing Test Data

In [30]:
df_test = pd.read_csv('data_test.csv', parse_dates=['time_exit', 'time_entry'])
df_test = df_test.drop_duplicates(subset='hash', keep='last')

In [32]:
len(df_test)

33515

In [33]:
df_test_new_entry = df_test['time_entry'].apply(unix_time_millis)
df_test_new_exit = df_test['time_exit'].apply(unix_time_millis)

In [34]:
df_test['time_entry'] = df_test_new_entry
df_test['time_exit'] = df_test_new_exit

In [38]:
X_test = df_test.iloc[:, [3,4,8,9]]

In [44]:
y_pred = classifier.predict(X_test.as_matrix())

AttributeError: 'numpy.ndarray' object has no attribute 'as_matrix'

In [97]:
n = np.array(y_pred)
np.amax(n)

1

In [98]:
df_final = df_test.iloc[:, [2]]

In [99]:
df_final['target'] = y_pred

/userhome/comp3314/c3314g15/anaconda3/envs/tf36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [100]:
df_final.columns=['id','target']

In [101]:
df_final.describe()

,target
count,33515.000000
mean,0.272147
std,0.445072
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [105]:
df_final
df_final.to_csv('result.csv', index=False)

In [106]:

df_final.head()

,id,target
4,traj_00032f51796fd5437b238e3a9823d13d_31_5,0
7,traj_000479418b5561ab694a2870cc04fd43_25_10,0
10,traj_000506a39775e5bca661ac80e3f466eb_29_5,1
13,traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_4,0
17,traj_00063a4f6c12e1e4de7d876580620667_3_4,0
